In [0]:
# Librerias para el tratamiento de los datos
import pandas as pd
import numpy as np

# Librerias para la graficación de los datost
import matplotlib.pyplot as plt
import seaborn as sns;
sns.set()
import plotly.express as px

# Permite que las graficas se generen a mayor resolucion
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Permite ignorar los warnings de la libreta al generar algunos modelos
import warnings
warnings.simplefilter('ignore')

# Establece un ancho de libreta mayor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Libreria necesaria para los gráficos interactivos
from ipywidgets import interact 
import plotly.express as px

# Librerias necesarias para el aprendizaje de modelos
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import GridSearchCV, train_test_split

#Libreria para reducción de dimensionalidad
from sklearn.decomposition import PCA

#Modelos
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

#Métricas
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc

#Medicion de tiempos
import time

# Pickle
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def show_results(y, y_pred):
    sns.heatmap(confusion_matrix(y, y_pred), square=True, annot=True, fmt='d', cbar=True, cmap=plt.cm.Blues)
    plt.ylabel('Clase real')
    plt.xlabel('Predicción');
    plt.gca().set_ylim(2.0, 0)
    plt.show()
    print("Resultados")
    print('Accuracy: {}'.format(round(accuracy_score(y, y_pred), 3)))

In [0]:
df = pd.read_csv('/content/drive/My Drive/DatathonCajamar2020/Modelar_UH2020.txt', sep="|", index_col='ID', encoding='utf-8')

In [0]:
list_class_order = ['RESIDENTIAL', 'PUBLIC', 'RETAIL', 'OFFICE', 'INDUSTRIAL', 'AGRICULTURE', 'OTHER']

In [0]:
list_preprocess_function = []

In [0]:
X = df.drop(columns=['CLASE']).copy()
y = df['CLASE'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=10)

df_modelar = X_train.merge(y_train, on='ID')
df_modelar_new = X_val.merge(y_val, on='ID')

print("Tamaño del conjunto de datos disponibles: ", len(df_modelar))
print("Tamaño del conjunto de nuevos datos: ", len(df_modelar_new))

Tamaño del conjunto de datos disponibles:  87745
Tamaño del conjunto de nuevos datos:  15485


In [0]:
df_modelar['mean_red'] = df_modelar.loc[:, 'Q_R_4_0_0':'Q_R_4_1_0'].mean(axis=1)
df_modelar['mean_green'] = df_modelar.loc[:, 'Q_G_3_0_0':'Q_G_3_1_0'].mean(axis=1)
df_modelar['mean_blue'] = df_modelar.loc[:, 'Q_B_2_0_0':'Q_B_2_1_0'].mean(axis=1)
df_modelar['mean_nir'] = df_modelar.loc[:, 'Q_NIR_8_0_0':'Q_NIR_8_1_0'].mean(axis=1)

In [0]:
df_modelar['std_red'] = df_modelar.loc[:, 'Q_R_4_0_0':'Q_R_4_1_0'].std(axis=1)
df_modelar['std_green'] = df_modelar.loc[:, 'Q_G_3_0_0':'Q_G_3_1_0'].std(axis=1)
df_modelar['std_blue'] = df_modelar.loc[:, 'Q_B_2_0_0':'Q_B_2_1_0'].std(axis=1)
df_modelar['std_nir'] = df_modelar.loc[:, 'Q_NIR_8_0_0':'Q_NIR_8_1_0'].std(axis=1)

In [0]:
CENTER_X = df_modelar['X'].sum()/len(df_modelar['X'])
CENTER_Y = df_modelar['Y'].sum()/len(df_modelar['Y'])

In [0]:
def distance(p0, p1):
  return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)

def process_distance_to_center(df):
  df_modelar['distance_to_center'] = df_modelar.apply(lambda r: distance([r['X'], r['Y']], [center_x, center_y]), axis=1)
  df_modelar['distance_to_center'] = df_modelar['distance_to_center'] / df_modelar['distance_to_center'].max() #Normalizamos 0,1

In [0]:
list_preprocess_function.append(process_distance_to_center)

In [0]:
def process_cadastralquality(value):
    dic = {'A': 11, 'B': 10, 'C': 9}
    if value in dic:
        return dic[value]
    else:
        try:
            return 9 - int(value)
        except ValueError:
            return np.nan
        
def process_cadastral(df):
    df['CADASTRALQUALITYID'] = df['CADASTRALQUALITYID'].apply(process_cadastralquality).astype(np.int64)

In [0]:
list_preprocess_function.append(process_cadastral)

In [0]:
print("\nPasos de preprocesamiento: ")
for step, function in enumerate(list_preprocess_function):
    print("\t {:d}: {:s}".format(step, function.__name__))

In [0]:
def preprocess_data(df, list_preprocess_function):
    for func in list_preprocess_function:
        func(df)
    return df

In [0]:
X = df_modelar[['X', 'Y']].copy()
y = df_modelar['CLASE'].copy()

w = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=10)
n = 5
neigh = KNeighborsClassifier(n_neighbors=n, weights='uniform')
neigh.fit(X, y)

In [0]:
pickle.dump(neigh, open('/content/drive/My Drive/knn_MBM.model', 'wb'))

In [0]:
def process_neihgbours(df):
  predict = neigh.predict_proba(df[['X', 'Y']])
  aux = pd.DataFrame(predict*n, columns=[ 'N_'+c for c in neigh.classes_ ], index=df.index)
  df = df.merge(aux, on='ID', how='inner')
  df.drop(columns=['X', 'Y'], inplace=True)

In [0]:
list_preprocess_function.append(process_neihgbours)

In [0]:
data = df_modelar.groupby('CLASE')
d = {}
aux_without_outliers = []
for name, group in data:
    d[name] = len(group[group['AREA']> group['AREA'].quantile(0.75)])
    group = group[group['AREA']< group['AREA'].quantile(0.75)]
    aux_without_outliers.append(group.reset_index())

In [0]:
df_aux_without_outliers = pd.concat(aux_without_outliers).set_index('ID').sample(frac=1, random_state=10)

In [0]:
df_aux_without_outliers['CLASE'].value_counts()

In [0]:
preprocess_data(df_modelar, list_preprocess_function)

In [0]:
number_df_columns = df_modelar.select_dtypes(include=np.number).columns.tolist()

print('\n- \033[1mVariables numéricas\033[0m: ',number_df_columns)

In [0]:
#Transformación que se le va a aplicar a las columnas numéricas
num_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
                             ('scaler', StandardScaler())])

In [0]:
#Transformación que se le aplicaría a las columnas categóricas, en el caso de que existan
cat_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                           ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [0]:
#Transformador que se aplica a cada una de las columnas en función a lo declarado previamente
df_modelar_trans = ColumnTransformer(transformers=[('num', num_transformer, number_df_columns)])

### Modelo Binario

In [0]:
df_modelar['SUBCLASE'] = (df_modelar['CLASE'] == 'RESIDENTIAL').astype(int)

In [0]:
#Para nuestro primer modelo vamos a usar todas las variables
X = df_modelar.drop(columns=['CLASE', 'SUBCLASE']).copy()
y = df_modelar['SUBCLASE'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=10)

In [0]:
rfc_p_model = xgb.XGBClassifier(random_state=10)
underSampling = RandomUnderSampler('majority', random_state=10)
smote = SMOTE('auto', random_state=10)

parameters = {
    'clas__max_depth': [10]
}

minsait_pipe_rfc_p = Pipeline(steps=[('prep', df_minsait_trans), ('smt', smote), ('clas', rfc_p_model)])

GSR_p = GridSearchCV(minsait_pipe_rfc_p, parameters, cv=5, n_jobs=-1, scoring='f1', refit=True, verbose=1)
GSR_p.fit(X_train, y_train)
    
print("Mejor score: ", GSR_p.best_score_)
print("Mejore configuración de parámetros: ", GSR_p.best_params_)

minsait_pipe_rfc_p = GSR_p.best_estimator_

In [0]:
y_pred = minsait_pipe_rfc_p.predict(X_test)
show_results(y_test, y_pred)
print(classification_report(y_test, y_pred))

In [0]:
importances = minsait_pipe_rfc_p['clas'].feature_importances_
# Extrae los índices ordenados de menor a mayor
ranking = np.argsort(importances)

#Ahora que hemos obtenido las importancias de las variables y su ranking, vamos a dibujarlo
plt.figure(figsize=(28,20))
plt.barh(range(df_modelar.shape[1]-2), importances[ranking],color=sns.diverging_palette(220, 20, n=55),align='center')
plt.yticks(range(df_modelar.shape[1]-2), X_test.columns[ranking], fontsize=9)
plt.show()

In [0]:
pickle.dump(minsait_pipe_rfc_p, open('/content/drive/My Drive/binary_xgboost_MBM.model', 'wb'))

### Modelo Multietiqueta

In [0]:
y_pred_train = minsait_pipe_rfc_p.predict(X_train)

In [0]:
X_multi = df_modelar.loc[X_train[y_pred_train==0].index.tolist()].drop(columns=['CLASE', 'SUBCLASE']).copy()
y_multi = df_modelar.loc[X_train[y_pred_train==0].index.tolist()]['CLASE'].copy()

In [0]:
y_multi.value_counts()

In [0]:
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_multi, y_multi, test_size=0.3, random_state=10)
#smt = SMOTE('not majority', random_state=10)

In [0]:
model = RandomForestClassifier(random_state=10)

parameters = {}
parameters['clas__max_depth'] = [5, 10, 20] #Quitamos None para que no sobreajuste al conjunto de train y que no pueda hacer la profundidad que el modelo desee. 
                                            # Lo forzamos a ponerle profundidad, como máximo 20, aunque se podría establecer mayor profundidad pero no queremos que sobreajute a los datos del train.
parameters['clas__n_estimators'] = [100, 200]
parameters['clas__class_weight'] = ['balanced']

pipe = Pipeline(steps=[('prep', df_minsait_trans),('clas', model)])
GS = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, scoring='accuracy', refit=True, verbose=1)
GS.fit(X_train_multi, y_train_multi)
print(f"Mejor score: ", GS.best_score_)
print("Mejore configuración de parámetros: ", GS.best_params_)

model_multilabel = GS.best_estimator_

In [0]:
y_pred_multi = model_multilabel.predict(X_test_multi)
print(classification_report(y_test_multi, y_pred_multi))

In [0]:
importances = model_multilabel['clas'].feature_importances_
# Extrae los índices ordenados de menor a mayor
ranking = np.argsort(importances)

#Ahora que hemos obtenido las importancias de las variables y su ranking, vamos a dibujarlo
plt.figure(figsize=(28,20))
plt.barh(range(df_modelar.shape[1]-2), importances[ranking],color=sns.diverging_palette(220, 20, n=55),align='center')
plt.yticks(range(df_modelar.shape[1]-2), X_test.columns[ranking], fontsize=9)
plt.show()

In [0]:
pickle.dump(model_multilabel, open('/content/drive/My Drive/multilabel_rfc_MBM.model', 'wb'))